# Debt Price Model & Market Model Validation

## Overview

The purpose of this experiment, is to validate the system model, using a debt market regression model trained using historical data.

## Shared setup and dependencies

In [1]:
from pathlib import Path
import os

path = Path().resolve()
root_path = str(path).split('notebooks')[0]
os.chdir(root_path)

In [2]:
from shared import *

In [3]:
%pip show cadCAD

Name: cadCAD
Version: 0.4.23
Summary: cadCAD: a differential games based simulation software package for research, validation, and         Computer Aided Design of economic systems
Home-page: https://github.com/cadCAD-org/cadCAD
Author: Joshua E. Jodesty
Author-email: joshua@block.science
License: LICENSE.txt
Location: /home/aclarkdata/repos/reflexer/venv/lib/python3.8/site-packages
Requires: fn, funcy, pathos, pandas
Required-by: 
Note: you may need to restart the kernel to use updated packages.


# Parameters

These are the key model parameters:
* `options.DebtPriceSource == options.DebtPriceSource.DEBT_MARKET_MODEL`, which is a historical data regression model for the debt market.

In [4]:
debt_price_data = load_debt_price_data(options.DebtPriceSource.DEBT_MARKET_MODEL.value)

/home/aclarkdata/repos/reflexer/venv/lib/python3.8/site-packages/pyparsing.py:3190: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)
Exception ignored in: <function AutoML.__del__ at 0x7f82793598b0>
Traceback (most recent call last):
  File "/home/aclarkdata/repos/reflexer/venv/lib/python3.8/site-packages/autosklearn/automl.py", line 1168, in __del__
    self._backend.context.delete_directories(force=False)
AttributeError: 'AutoMLRegressor' object has no attribute '_backend'


ModuleNotFoundError: No module named 'autosklearn.pipeline.components.regression.ridge_regression'

In [ ]:
debt_price_data

* PI controller constant sweep

In [ ]:
kp_sweep = [-1.5e-6]
ki_sweep = [lambda control_period=3600: 0] #-1e-6 / control_period


* Controller enabled

In [ ]:
controller_enabled = [False, True]

* Error term calculation

In [ ]:
error_term = [
    lambda target, measured: target - measured,
]

* Integral type

In [ ]:
integral_type = [options.IntegralType.LEAKY.value]

# Simulation Configuration

Set the simulation timesteps to the minimum dataset length:

In [ ]:
minimum_timesteps = min([df.shape[0] for df in debt_price_data])
SIMULATION_TIMESTEPS = range(minimum_timesteps)
SIMULATION_TIMESTEPS

Override certain parameter sweeps with the generated sets, using ConfigWrapper for convenience:

In [ ]:
# Update parameter options
update_params = {
    # By using an Enum, we can self-document all possible options
    options.DebtPriceSource.__name__: [options.DebtPriceSource.DEBT_MARKET_MODEL.value],
    options.IntegralType.__name__: integral_type,
    'controller_enabled': controller_enabled,
    'expected_control_delay': [lambda _timestep: 0], # 1200 # stochastic process
    'minumum_control_period': [lambda _timestep: 3600],
    # Generate a lambda for each test dataframe, that returns a row value at a specific timestep
    'seconds_passed': [
        lambda timestep, df=df.copy(): int(df.iloc[timestep - 1]['seconds_passed'])
        for df in debt_price_data
    ],
    'price_move': [
        lambda timestep, df=df.copy(): float(df.iloc[timestep - 1]['price_move'])
        for df in debt_price_data
    ],
    'delta_output': [
        lambda state, timestep: (0.1 if state == 'debt_price' and timestep == 50 else 0) \
            + (-0.2 if state == 'debt_price' and timestep == 250 else 0)
    ],
    'kp': kp_sweep,
    'ki': ki_sweep,
    # Select or sweep the error term calculation, as a lambda
    # e.g. p*-p vs (p*-p)/p vs (p*-p)/p*
    'error_term': error_term
}

update_initial_state = {
    'target_price': 1.0,
    'market_price': 1.0,
    'debt_price': 1.0,
}

'''
The ConfigWrapper allows you to pass a model as an argument, and update the simulation configuration.
Maps (params, states) would be merge updated, and all other options are overrides.
'''
system_simulation = ConfigWrapper(system_model_v1, M=update_params, initial_state=update_initial_state, T=SIMULATION_TIMESTEPS)

# Simulation Execution

In [ ]:
del configs[:]

system_simulation.append()

(data, tensor_field, sessions) = run(drop_midsteps=True)

# Data Preparation

In [ ]:
df = data.copy()
df

In [ ]:
# Export CSV market_price dataset
df.query('subset == 1')['market_price'].to_csv('tests/data/regression_market_prices.csv', header=True)

In [ ]:
df['target_rate_seconds'] = df.target_rate # * 3600
df['error_star_derivative_scaled'] = df.error_star_derivative * 3600
df['error_hat_derivative_scaled'] = df.error_star_derivative * 3600

# Data Analysis

In [ ]:
fig = px.line(
    df,
    x='timestamp',
    y=['debt_price', 'target_price', 'market_price'],
    facet_col='subset',
    facet_col_wrap=2,
    template='seaborn',
    height=800
)

fig.show()
fig.write_image('plots/controller-target-plot.png', width=1000, height=850)

In [ ]:
fig = px.line(
    df[df.subset == 0],
    x='timestamp',
    y=['debt_price'],
    facet_col='subset',
    facet_col_wrap=2,
    template='seaborn',
    height=800
)

# fig.show()
fig.write_image('plots/debt-price-plot.svg', width=1000, height=850)
from IPython.display import SVG,display
display(SVG(filename='plots/debt-price-plot.svg'))

In [ ]:
fig = px.line(
    df,
    x='timestamp',
    y=['error_star','error_star_derivative_scaled'],
    facet_col='subset',
    facet_col_wrap=2,
    template='seaborn',
    height=800
)

# fig.show()
fig.write_image('plots/error-star-plot.svg', width=1000, height=850)
from IPython.display import SVG,display
display(SVG(filename='plots/error-star-plot.svg'))

In [ ]:
from utils.plots import integral_plot

integral_plot(df[df.timestep<100])

## Conclusion